In [1]:
from conda_skeletor.main import split_deps, get_runtime_deps, construct_template_info, package_mapping
from conda_skeletor import setup_parser
import depfinder
import os
import yaml
import re
import pandas as pd

The module located at /home/edill/miniconda/lib/python3.4/site-packages/whatsmyversion-0.0.4.post10-py3.4.egg/whatsmyversion.py does not have a git repo in its directory hierarchy
INFO:whatsmyversion:The module located at /home/edill/miniconda/lib/python3.4/site-packages/whatsmyversion-0.0.4.post10-py3.4.egg/whatsmyversion.py does not have a git repo in its directory hierarchy
path = /home/edill/miniconda/lib/python3.4/site-packages/whatsmyversion-0.0.4.post10-py3.4.egg/whatsmyversion.py
DEBUG:whatsmyversion:path = /home/edill/miniconda/lib/python3.4/site-packages/whatsmyversion-0.0.4.post10-py3.4.egg/whatsmyversion.py
split_path = ['', 'home', 'edill', 'miniconda', 'lib', 'python3.4', 'site-packages', 'whatsmyversion-0.0.4.post10-py3.4.egg', 'whatsmyversion.py']
DEBUG:whatsmyversion:split_path = ['', 'home', 'edill', 'miniconda', 'lib', 'python3.4', 'site-packages', 'whatsmyversion-0.0.4.post10-py3.4.egg', 'whatsmyversion.py']
version_info_folder = whatsmyversion-0.0.4.post10-py3.4.eg

In [2]:
# Function coped from main() --> conda-skeletor.main:main
def find_imports(path_to_source):
    # find the dependencies for all modules in the source directory
    repo_deps = depfinder.iterate_over_library(path_to_source)

    # Compile the regexers listed in the conda-skeleton.yml
    test_regexers = [re.compile(reg) for reg in skeletor_config.get('test_regex', [])]
    ignore_path_regexers = [re.compile(reg) for reg in skeletor_config.get('ignore_path_regex', [])]
    include_path_regexers = [re.compile(reg) for reg in skeletor_config.get('include_path_regex', [])]
    ignored, without_ignored = split_deps(repo_deps, ignore_path_regexers)
    included, without_included = split_deps(without_ignored, include_path_regexers)
    tests, without_tests = split_deps(included, test_regexers)
    
    return tests, without_tests

In [3]:
skeletor_config = yaml.load(open('conda-skeletor.yml', 'r'))

In [4]:
skeletor_config

{'blacklist_packages': ['setupext',
  'versioneer',
  'ctrans',
  'skxray',
  'src',
  'netCDF4',
  'pyFAI'],
 'ignore_path_regex': ['.*(doc).*', '.*(examples).*', '.*(versioneer).*'],
 'include_path_regex': ['.*(skxray).*'],
 'test_regex': ['.*(test).*']}

In [5]:
path_to_source = os.path.join(os.path.expanduser('~/dev/python/scikit-xray'))

In [6]:
test, without_test = find_imports(path_to_source)

...................................................................................

In [7]:
def find_lib_name(without_test):
    non_test_paths = [full_path for mod_name, full_path, catcher in without_test]
    non_test_paths = [path.split(os.sep) for path in non_test_paths]
    df = pd.DataFrame(non_test_paths)
    prev_col_name = None
    equivalent = True
    for col_name in df:
        equivalent = not any([val != df[col_name][0] for val in df[col_name]])
        if not equivalent:
            break
        prev_col_name = col_name
    return df[prev_col_name][0]

In [40]:
lib_name = find_lib_name(without_test)

In [89]:
prev_col_name

6

In [10]:
path = without_test[0][1]

In [39]:
lib_name

'__init__'

In [45]:
def into_importable(full_module_path, lib_name):
    relative_module_path = full_module_path.split(lib_name, maxsplit=1)[1]
    # trim the '.py'
    relative_module_path = relative_module_path[:-3]
    # add the library name back in
    relative_module_path = lib_name + relative_module_path
    importable_path = '.'.join(relative_module_path.split(os.sep))
    # turn imports from 'foo.bar.baz.__init__' into 'foo.bar.baz'
    if importable_path.endswith('__init__'):
        importable_path = importable_path[:-9]
    # turn the string from path/to/module to path.to.module
    return importable_path

In [47]:
all_imports = [into_importable(path_to_module, lib_name) for mod_name, path_to_module, catcher in without_test]

'skxray.fluorescence'